In [ ]:
%load_ext autoreload
%autoreload 2

# Find duplicates across ROIs

Look for cells that are detected on multiple consecutive ROIs

In [ ]:
import iss_preprocess as issp
import iss_analysis as issa
from iss_analysis.barcodes import barcodes as bar
from iss_analysis.barcodes.diagnostics import (
    plot_gmm_clusters,
    plot_error_along_sequence,
)
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
project = "becalia_rabies_barseq"
mouse = "BRAC8498.3e"

error_correction_ds_name = "BRAC8498.3e_error_corrected_barcodes_16"
data_path = f"{project}/{mouse}"
analysis_folder = issp.io.get_processed_path(data_path) / "analysis"


# Get rabies data

In [ ]:
(
    rab_spot_df,
    rab_cells_barcodes,
    rab_cells_properties,
) = issa.segment.get_barcode_in_cells(
    project,
    mouse,
    error_correction_ds_name,
    valid_chambers=None,
    save_folder=None,
    verbose=True,
)
# find starter
starters_positions = issa.io.get_starter_cells(project, mouse)
rabies_cell_properties = issa.segment.match_starter_to_barcodes(
    project,
    mouse,
    rab_cells_properties,
    rab_spot_df,
    starters=starters_positions,
    redo=False,
)
rabies_cell_properties.head()


# Get ARA coordinates of spots

In [ ]:
for (chamber, roi), spots in rab_spot_df.groupby(["chamber", "roi"]):
    data_path = f"{project}/{mouse}/{chamber}"
    sp = issp.pipeline.ara_registration.spots_ara_infos(data_path, spots, roi, atlas_size=10, acronyms=False, inplace=False)
    for col in ['ara_x', 'ara_y', 'ara_z', 'area_id']:
        rab_spot_df.loc[spots.index, col] = sp[col]



In [ ]:
# same for cells
for (chamber, roi), cell_prop in rabies_cell_properties.groupby(["chamber", "roi"]):
    data_path = f"{project}/{mouse}/{chamber}"
    sp = issp.pipeline.ara_registration.spots_ara_infos(data_path, cell_prop, roi, atlas_size=10, acronyms=False, inplace=False)
    for col in ['ara_x', 'ara_y', 'ara_z', 'area_id']:
        rabies_cell_properties.loc[cell_prop.index, col] = sp[col]


In [ ]:
print(chamber, roi)
fig = plt.figure(figsize=(10, 10))  
ax = fig.add_subplot(1,1,1, aspect='equal')
sp = rab_spot_df.query("chamber == @chamber & roi == @roi")
ax.scatter(sp.x, sp.y, c=sp.area_id%20, cmap="tab20", vmin=0, vmax=20)
cells = rabies_cell_properties.query("chamber == @chamber & roi == @roi")
ax.scatter(cells.x, cells.y, c=cells.area_id%20, cmap="tab20", marker="x", s=100, vmin=0, vmax=20)